In [ ]:
!pip install TenSeal
!pip install deepface

In [2]:
import tenseal as ts
from deepface import DeepFace

In [3]:
import base64

In [31]:
import math

# Embeddings
Client side

In [4]:
path_img1="/content/image1.jpg"
path_img2="/content/image2.jpg"


In [5]:
img1_embed=DeepFace.represent(path_img1,model_name='Facenet')[0]["embedding"]
img2_embed=DeepFace.represent(path_img2,model_name='Facenet')[0]["embedding"]

In [6]:
len(img1_embed)

128

# To read and write byte objects

In [7]:
def write_data(file_name,file_content):
  if type(file_content)==bytes:

    #Bytes to b64

    file_content=base64.b64encode(file_content)

  with open(file_name,'wb') as f:
    f.write(file_content)  #Will write byte object into b64

def read_data(file_name):
  with open(file_name,'rb') as f:
    file_content=f.read()

  return base64.b64decode(file_content)    #this will return bytes  

# Initialize

client side because keys are generated

In [8]:
context=ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192,coeff_mod_bit_sizes=[60,40,40,60])

#CKKS because the items are real numbers
#Context stores both public and private key

In [9]:
context.generate_galois_keys()
context.global_scale=2**40

In [10]:
secret_context=context.serialize(save_secret_key=True)#Since tenseal doesn't support pickle, we will have to convert to bytes

In [11]:
type(secret_context) #see that it is now bytes

bytes

In [12]:
write_data(file_name="secret.txt",file_content=secret_context) #content of secret/private key

In [13]:
context.make_context_public()#to drop the private key else it will be leaked
public_context=context.serialize()#not setting save secret key to true it is false by default and we don't want to save it
write_data(file_name="public.txt",file_content=public_context)

In [14]:
del context,secret_context,public_context

#We require private key for encryption but since it is already dropped we will restore the context and hence deleting contexts now

# Encryption

client side; results will go to server

In [15]:
context = ts.context_from(read_data("secret.txt"))
type(context)

tenseal.enc_context.Context

In [16]:
enc_v1 = ts.ckks_vector(context, img1_embed)
enc_v2 = ts.ckks_vector(context, img2_embed)

In [17]:
write_data(file_name="enc_v1.txt",file_content=enc_v1.serialize())
write_data(file_name="enc_v2.txt",file_content=enc_v2.serialize())

In [18]:
del context,enc_v1,enc_v2

# Calculations

cloud side

In [19]:
context=ts.context_from(read_data("public.txt"))

In [20]:
enc_v1=ts.lazy_ckks_vector_from(read_data(file_name='enc_v1.txt'))
enc_v2=ts.lazy_ckks_vector_from(read_data(file_name='enc_v2.txt'))

In [21]:
enc_v1.link_context(context)
enc_v2.link_context(context)


In [22]:
euclidean_squared=enc_v1-enc_v2
euclidean_squared=euclidean_squared.dot(euclidean_squared) #performed on encrypted data

In [23]:
write_data(file_name="euclidean_squared.txt",file_content=euclidean_squared.serialize())

In [25]:
del context,enc_v1,enc_v2,euclidean_squared

# Decryption

In [28]:
context=ts.context_from(read_data("secret.txt"))

In [29]:
euclidean_squared=ts.lazy_ckks_vector_from(read_data(file_name='euclidean_squared.txt'))
euclidean_squared.link_context(context)

In [33]:
euclidean_squared=math.sqrt(euclidean_squared.decrypt()[0])

In [34]:
if euclidean_squared<10:
  print("same person")
else:
  print("different person")  

same person
